### Downloading Dataset

So we will first need to download the python library by AIcrowd that will allow us to download the dataset by just inputting the API key. 

In [1]:
%%capture
!pip install -qq aicrowd-cli
%load_ext aicrowd.magic

#!pip install -qU wandb

%aicrowd login --api-key ba03f6555248226a41217fd4e9246e65

!rm -rf dataset
!mkdir dataset
%aicrowd ds dl -c mask-prediction -o dataset

!unzip dataset/train.zip -d dataset/ > /dev/null
!unzip dataset/val.zip -d dataset/ > /dev/null
!unzip dataset/test.zip -d dataset/ > /dev/null

# Importing Libraries

In [2]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
import seaborn as sns

In [3]:
dim = 512
fold = 0 # doing with fold 0

# Diving in the dataset 🕵️‍♂️

In [4]:
train_images = './dataset/train'
val_images = './dataset/val'
test_images = './dataset/test'

In [5]:
train_df = pd.read_csv("./dataset/train.csv")
val_df = pd.read_csv("./dataset/val.csv")

test_df = pd.read_csv("./dataset/sample_submission.csv")

In [6]:
train_df.shape, val_df.shape, test_df.shape

In [7]:
def get_train_path(image_id):
  return f'dataset/train/{image_id}.jpg'

def get_valid_path(image_id):
  return f'dataset/val/{image_id}.jpg'

def get_test_path(image_id):
  return f'dataset/test/{image_id}.jpg'

In [8]:
train_df['image_path'] = train_df['ImageID'].apply(get_train_path)
val_df['image_path'] = val_df['ImageID'].apply(get_valid_path)
test_df['image_path'] = test_df['ImageID'].apply(get_test_path)

In [9]:
test_df.head(5)

# yolov5 stuff

In [10]:
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5') # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# Inference

In [11]:
weights_dir = '../input/mask-pred-train/yolov5/runs/train/exp/weights/best.pt'

In [12]:
%cd ../

In [13]:
########### no need to run this ###############

# os.mkdir('yolov5/tmp')
# os.mkdir('yolov5/tmp/weights')

#!rm -r 'yolov5/tmp'

In [14]:
shutil.copytree('../input/mask-pred-test-files/test', 'yolov5/tmp/images')
shutil.copy(weights_dir, 'yolov5/tmp/weights')

In [15]:
%cd yolov5

In [16]:
!python detect.py --weights 'tmp/weights' --img 512\
--conf 0.9 --iou 0.4 --source 'tmp/images' --save-txt --save-conf --exist-ok

# Plot

In [17]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import random
import cv2
from glob import glob
from tqdm import tqdm

files = glob('runs/detect/exp/*jpg')
for _ in range(3):
    row = 4
    col = 4
    grid_files = random.sample(files, row*col)
    images     = []
    for image_path in tqdm(grid_files):
        img          = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        images.append(img)

    fig = plt.figure(figsize=(col*5, row*5))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(col, row),  # creates 2x2 grid of axes
                     axes_pad=0.05,  # pad between axes in inch.
                     )

    for ax, im in zip(grid, images):
        # Iterating over the grid returns the Axes.
        ax.imshow(im)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

# Submission

In [18]:
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [19]:
test_df.head(2)

In [20]:
test_df['width'] = 512
test_df['height'] = 512

In [37]:
image_ids = []
PredictionStrings = []
class_ids = []
bbox = []

for file_path in tqdm(glob('runs/detect/exp/labels/*txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.ImageID==image_id,['width', 'height']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    c = data[:, 0]
    #bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
    #print(bboxes)
    bboxes = np.round(yolo2voc(h, w, data[:, 2:]))
#     for idx in range(len(bboxes)):
#         bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    class_ids.append(c)
    image_ids.append(image_id)
    PredictionStrings.append(bboxes)

In [52]:
pred_df = pd.DataFrame({'image_id':image_ids,
                        'class_id' : class_ids,
                        'bbox':PredictionStrings})

In [57]:
pred_df.head(5)

In [54]:
pred_df['class_id'] = pred_df['class_id'].apply(lambda x : int(x[0]))
pred_df['bbox'] = pred_df['bbox'].apply(lambda x : list(x[0]))

In [67]:
pred_df.head(5)